## [2] Preprocessing

[2.1.3] Removing Stop Word

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')

def remove_stopwords(text):
    filtered_words = [word for word in text if word.lower() not in stopwords.words('english')]
    return ' '.join(filtered_words)

df['Email Text'] = df['Email Text'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyboardInterrupt: 

In [ ]:
filepath='/content/Dataset/Prepro.csv'
df.to_csv(filepath,index=False)

## [3] Feature Engineering

[3.1] Word embedding

In [ ]:
df = pd.read_csv('/content/Dataset/Prepro.csv')
df

,Unnamed: 0,Email Text,Email Type
0,0,re 6 1100 disc uniformitarianism re 1086 sex l...,Safe Email
1,1,the other side of galicismos galicismo is a sp...,Safe Email
2,2,re equistar deal tickets are you still availab...,Safe Email
3,3,hello i am your hot lil horny toy i am the one...,Phishing Email
4,4,software at incredibly low prices 86 lower dra...,Phishing Email
...,...,...,...
18645,18646,date a lonely housewife always wanted to date ...,Phishing Email
18646,18647,request submitted access request for anita dup...,Safe Email
18647,18648,re important prc mtg hi dorn john as you disco...,Safe Email
18648,18649,press clippings letter on californian utilitie...,Safe Email


In [ ]:
from gensim.models import Word2Vec,KeyedVectors
from tqdm import tqdm
import multiprocessing
from nltk.tokenize import word_tokenize

In [ ]:
cores=multiprocessing.cpu_count()

Tokenization

In [ ]:
import nltk
nltk.download('punkt')

# Convert non-string values to empty strings
df['Email Text'] = df['Email Text'].apply(lambda x: '' if not isinstance(x, str) else x)

# Tokenize text in the 'Email Text' column
df['tokens'] = df['Email Text'].apply(lambda x: word_tokenize(x))
df['tokens']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0        [re, 6, 1100, disc, uniformitarianism, re, 108...
1        [the, other, side, of, galicismos, galicismo, ...
2        [re, equistar, deal, tickets, are, you, still,...
3        [hello, i, am, your, hot, lil, horny, toy, i, ...
4        [software, at, incredibly, low, prices, 86, lo...
                               ...                        
18645    [date, a, lonely, housewife, always, wanted, t...
18646    [request, submitted, access, request, for, ani...
18647    [re, important, prc, mtg, hi, dorn, john, as, ...
18648    [press, clippings, letter, on, californian, ut...
18649                                              [empty]
Name: tokens, Length: 18650, dtype: object

In [ ]:
df

,Unnamed: 0,Email Text,Email Type,tokens
0,0,re 6 1100 disc uniformitarianism re 1086 sex l...,Safe Email,"[re, 6, 1100, disc, uniformitarianism, re, 108..."
1,1,the other side of galicismos galicismo is a sp...,Safe Email,"[the, other, side, of, galicismos, galicismo, ..."
2,2,re equistar deal tickets are you still availab...,Safe Email,"[re, equistar, deal, tickets, are, you, still,..."
3,3,hello i am your hot lil horny toy i am the one...,Phishing Email,"[hello, i, am, your, hot, lil, horny, toy, i, ..."
4,4,software at incredibly low prices 86 lower dra...,Phishing Email,"[software, at, incredibly, low, prices, 86, lo..."
...,...,...,...,...
18645,18646,date a lonely housewife always wanted to date ...,Phishing Email,"[date, a, lonely, housewife, always, wanted, t..."
18646,18647,request submitted access request for anita dup...,Safe Email,"[request, submitted, access, request, for, ani..."
18647,18648,re important prc mtg hi dorn john as you disco...,Safe Email,"[re, important, prc, mtg, hi, dorn, john, as, ..."
18648,18649,press clippings letter on californian utilitie...,Safe Email,"[press, clippings, letter, on, californian, ut..."


In [ ]:
word2vec_model = Word2Vec(df['tokens'], vector_size=100, window=5, min_count=1, sg=1)

In [ ]:
def text_to_vector(tokens):
    vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return None

In [ ]:
df['Vector'] = df['tokens'].apply(text_to_vector)

In [ ]:
df = df.dropna()

In [ ]:
filepath='/content/Dataset/Feature_Eng.csv'
df.to_csv(filepath,index=False)

## [4] Modeling

SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ImportError: cannot import name 'SVM' from 'sklearn.svm' (/usr/local/lib/python3.10/dist-packages/sklearn/svm/__init__.py)

In [ ]:
X = pd.DataFrame(df['Vector'].tolist())
y = df['Email Type']

y = y.replace({'Phishing Email': 0, 'Safe Email': 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)


SVC(kernel='linear', random_state=42)

Evaluation

In [ ]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9551918433056077


In [ ]:
from sklearn.metrics import f1_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')

# แสดง classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.9551918433056077
F1 Score: 0.9621915327145122
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      1477
           1       0.98      0.94      0.96      2250

    accuracy                           0.96      3727
   macro avg       0.95      0.96      0.95      3727
weighted avg       0.96      0.96      0.96      3727

